In [75]:
# importing neccessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score,confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

In [76]:
full_data=pd.read_csv('waterquality.csv')
full_data.head()

,STATION CODE,LOCATIONS,STATE,TEMP,DO,pH,CONDUCTIVITY,BOD,NITRATE_N_NITRITE_N,FECAL_COLIFORM,TOTAL_COLIFORM
0,1312,"GODAVARI AT JAYAKWADI DAM, AURNAGABAD,MAHARASHTRA",MAHARASHTRA,29.2,6.4,8.1,735.0,3.4,2.00,3.0,73.0
1,2177,GODAVARI RIVER NEAR SOMESHWAR TEMPLE.,MAHARASHTRA,24.5,6.0,8.0,270.0,3.1,2.00,72.0,182.0
2,2182,GODAVARI RIVER AT SAIKHEDA.,MAHARASHTRA,25.8,5.5,7.8,355.0,4.2,9.00,59.0,133.0
3,2179,"GODAVARI RIVER AT HANUMAN GHAT, NASHIK CITY.",MAHARASHTRA,24.8,5.5,7.8,371.0,5.6,3.55,90.0,283.0
4,2183,GODAVARI RIVER AT NANDUR- MADMESHWAR DAM.,MAHARASHTRA,25.7,5.7,7.9,294.0,3.2,2.69,45.0,132.0


In [77]:
object_cols = [col for col in list(full_data.columns) if full_data[col].dtype == "object"]
print(object_cols)

['LOCATIONS', 'STATE']


In [78]:
full_data.drop(labels=object_cols,axis=1, inplace= True)

In [79]:
full=full_data

In [80]:
def imputer(full):
    imput=SimpleImputer(strategy='mean')
    data_to_prepro=pd.DataFrame(imput.fit_transform(full))
    data_to_prepro.columns=full.columns
    return data_to_prepro

In [81]:
data_to_preprocess=imputer(full)

In [82]:
def preprocess(data):
    
    data['ndo']=data.DO.apply(lambda x:(100 if (x>=6)  
                                 else(80 if  (6>=x>=5.1) 
                                      else(60 if (5>=x>=4.1)
                                          else(40 if (4>=x>=3) 
                                              else 0)))))
    data['npH']=data.pH.apply(lambda x: (100 if (8.5>=x>=7)  
                                 else(80 if  (8.6>=x>=8.5) or (6.9>=x>=6.8) 
                                      else(60 if (8.8>=x>=8.6) or (6.8>=x>=6.7) 
                                          else(40 if (9>=x>=8.8) or (6.7>=x>=6.5)
                                              else 0)))))
    data['nco']=data.FECAL_COLIFORM .apply(lambda x:(100 if (5>=x>=0)  
                                 else(80 if  (50>=x>=5) 
                                      else(60 if (500>=x>=50)
                                          else(40 if (10000>=x>=500) 
                                              else 0)))))
    data['nbdo']=data.BOD.apply(lambda x:(100 if (3>=x>=0)  
                                 else(80 if  (6>=x>=3) 
                                      else(60 if (80>=x>=6)
                                          else(40 if (125>=x>=80) 
                                              else 0)))))
    data['nec']=data.CONDUCTIVITY.apply(lambda x:(100 if (75>=x>=0)  
                                 else(80 if  (150>=x>=75) 
                                      else(60 if (225>=x>=150)
                                          else(40 if (300>=x>=225) 
                                              else 0)))))
    data['nna']=data.NITRATE_N_NITRITE_N.apply(lambda x:(100 if (20>=x>=0)  
                                 else(80 if  (50>=x>=20) 
                                      else(60 if (100>=x>=50)
                                          else(40 if (200>=x>=100) 
                                              else 0)))))
    data['wph']=data.npH * 0.165
    data['wdo']=data.ndo * 0.281
    data['wbdo']=data.nbdo * 0.234
    data['wec']=data.nec* 0.009
    data['wna']=data.nna * 0.028
    data['wco']=data.nco * 0.281
    data['WQI']=data.wph+data.wdo+data.wbdo+data.wec+data.wna+data.wco
    data['WaterQuality']=data.WQI.apply(lambda x:(1 if (50>=x>=0) else 0) )
    #data ['WaterQuality']= data ['WaterQualitys'].map({'Unsuitable':0,'Excellent':1})
    #data.drop(labels='WaterQualitys',axis=1,inplace=True)
    features=['STATION CODE', 'TEMP', 'DO', 'pH', 'CONDUCTIVITY', 'BOD',
       'NITRATE_N_NITRITE_N', 'FECAL_COLIFORM', 'TOTAL_COLIFORM','WaterQuality']
    data_c=data[features]
    #full_train_data=data_c.copy()
    #X=full_train_data[features]
    #y=full_train_data['WaterQuality']
    #x_train,x_test,y_train,y_test=train_test_split(X,y,test_size=0.2, random_state=32,shuffle=True, stratify=y)
    #scale= StandardScaler()
   # scale.fit(X)
    #scaled=scale.transform(X)
    
    return data_c


In [83]:
preprocessed_data=preprocess(data_to_preprocess)
preprocessed_data[29:300]

,STATION CODE,TEMP,DO,pH,CONDUCTIVITY,BOD,NITRATE_N_NITRITE_N,FECAL_COLIFORM,TOTAL_COLIFORM,WaterQuality
29,2370.0,26.6,5.8,6.9,290.00000,1.2,2.40,48.000000,628.000000,0
30,14.0,27.0,6.2,7.7,380.00000,1.2,1.56,7384.170354,170.000000,0
31,1218.0,26.6,6.1,7.4,480.00000,1.3,1.98,3.000000,432.000000,0
32,2371.0,26.8,5.9,7.0,315.00000,1.2,1.29,23.000000,1100.000000,0
33,1484.0,17.5,8.5,8.1,684.97619,0.7,0.00,7384.170354,124396.967677,0
34,1485.0,16.3,8.8,8.1,684.97619,0.7,0.00,7384.170354,124396.967677,0
35,1486.0,16.7,8.1,8.0,684.97619,1.0,0.00,7384.170354,124396.967677,0
36,1487.0,17.3,8.4,8.1,684.97619,1.0,0.00,7384.170354,124396.967677,0
37,1488.0,17.0,8.7,8.1,684.97619,0.9,0.00,7384.170354,124396.967677,0
38,1489.0,17.0,8.2,8.0,684.97619,1.1,0.00,7384.170354,124396.967677,0


In [84]:
def splitting(data1):
    features=['STATION CODE', 'TEMP', 'DO', 'pH', 'CONDUCTIVITY', 'BOD',
       'NITRATE_N_NITRITE_N', 'FECAL_COLIFORM', 'TOTAL_COLIFORM']

    full_train_data=data1
    X1=full_train_data[features]
    y=full_train_data['WaterQuality']
    x_train,x_test,y_train,y_test=train_test_split(X1,y,test_size=0.25, shuffle=True, stratify=y, random_state=32)
    return x_train,x_test,y_train,y_test

In [85]:
x_train,x_test,y_train,y_test=splitting(preprocessed_data)

In [86]:

print(x_train.shape)

(400, 9)


In [87]:
model=RandomForestClassifier(max_depth=8, n_jobs=-1, random_state=42)

In [88]:
#pipeline =Pipeline([('scaler',StandardScaler()),('model',model)])

In [89]:
#pipeline.fit (x_train,y_train)

In [90]:
predictions=pipeline.predict(x_test)
predictions

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0], dtype=int64)

In [91]:
accuracy=pipeline.score(x_test,y_test)
print(f'The accuracy of the model is {accuracy}')

The accuracy of the model is 0.9925373134328358


In [92]:
confusionMatrix=confusion_matrix(y_test,predictions)
confusionMatrix

array([[128,   0],
       [  1,   5]], dtype=int64)

In [153]:
def predict_WQ(req, model):
    x_train,x_test,y_train,y_test=splitting(preprocessed_data)
    pipeline =Pipeline([('scaler',StandardScaler()),('model',model)])
    pipeline.fit (x_train,y_train)
    if type(req) == list:
        data_array=np.array(req).T
        trial=pd.DataFrame([req],columns=['STATION CODE', 'TEMP', 'DO', 'pH', 'CONDUCTIVITY', 'BOD',
       'NITRATE_N_NITRITE_N', 'FECAL_COLIFORM','TOTAL_COLIFORM'])
        trial
    else:
        trial=pd.DataFrame(req)
    y_pred =pipeline.predict(trial)
    if y_pred==1:
        pred=print('suitable')
    else:
        pred=print('unsuitable')
    return pred

In [166]:
tri1=[786, 34, 98, 8,0.98, 78,0.2, 9.7,8.6]


In [161]:
tri2=[2559.0,25.6,2.1,6.9,461.00000,8.6,0.00,3867.000000,14367.000000]


In [ ]:
tri3=[1060.0,17.2,8.7,7.7,684.97619,1.1,0.00,7384.170354,22.000000]

In [162]:
import pickle

##dump the model into a file
with open("modella.bin", 'wb') as f_out:
    pickle.dump(model, f_out) # write final_model in .bin file
    f_out.close()  # close the file 


In [163]:
with open('modella.bin', 'rb') as f_in:
    modell= pickle.load(f_in)

In [164]:
predict_WQ(tri1, modell)

suitable


In [167]:
predict_WQ(tri2, modell)

unsuitable


In [169]:
predict_WQ(tri3, modell)

unsuitable


pip install requests


In [100]:
import requests
url="http://127.0.0.1:9696/"
r=requests.post(url,json=tri1)
r.text.strip()

'<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 3.2 Final//EN">\n<title>405 Method Not Allowed</title>\n<h1>Method Not Allowed</h1>\n<p>The method is not allowed for the requested URL.</p>'

In [ ]:
x_train.describe().T